![header](../figures/logos_partenaires_DC_WOC-ESA.jpg)

# Agulhas: Benchmark MIOST plot maps
 

***
**Authors:** Datlas <br>
**Copyright:** 2023 Datlas <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h1><center>Agulhas: Benchmark of MIOST plot maps</center></h1>
<h4><center> The notebook aims to plot maps produced by the MIOST system in the Agulhas region. </center></h4> 
</div>

In [1]:
from glob import glob
import numpy as np
import os

In [2]:
import sys
sys.path.append('..')
from src.mod_plot import *
from src.mod_stat import *
from src.mod_spectral import *
from src.mod_interp import *
from src.mod_switchvar import *

In [3]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

<div class="alert alert-info" role="alert">

<h2>0. Parameters</h2>

</div>

In [4]:
time_min = '2019-01-15'                                        # time min for analysis
time_max = '2019-12-15'                                        # time max for analysis
output_dir = '../results'                                      # output directory path
os.system(f'mkdir -p {output_dir}')

# Region details
region = 'Mediterranean'
lon_min = -5                                          # domain min longitude
lon_max = 25                                          # domain max longitude
lat_min = 35.                                          # domain min latitude
lat_max = 47.                                          # domain max latitude

box_lonlat = {'lon_min':lon_min,'lon_max':lon_max,'lat_min':lat_min,'lat_max':lat_max}

method_name = 'MIOST'



<div class="alert alert-info" role="alert">

<h2>1. Input files</h2>

</div>

## Sea Surface Height maps to evaluate

In [5]:
%%time
list_of_maps = sorted(glob('../data/Mediterranean/products_Mediterranean/MIOST_Mediterranean.nc'))
ds_maps = xr.open_mfdataset(list_of_maps, combine='nested', concat_dim='time')
ds_maps = ds_maps.sel(time=slice(time_min, time_max))
ds_maps

CPU times: user 39.6 ms, sys: 10.9 ms, total: 50.5 ms
Wall time: 50.6 ms


<xarray.Dataset>
Dimensions:    (longitude: 256, latitude: 130, time: 335)
Coordinates:
  * longitude  (longitude) float64 0.0 0.1 0.2 0.3 0.4 ... 25.2 25.3 25.4 25.5
  * latitude   (latitude) float32 34.55 34.65 34.75 34.85 ... 47.25 47.35 47.45
  * time       (time) datetime64[ns] 2019-01-15 2019-01-16 ... 2019-12-15
Data variables:
    sla        (time, latitude, longitude) float64 dask.array<chunksize=(335, 130, 256), meta=np.ndarray>
    ugosa      (time, latitude, longitude) float64 dask.array<chunksize=(335, 130, 256), meta=np.ndarray>
    vgosa      (time, latitude, longitude) float64 dask.array<chunksize=(335, 130, 256), meta=np.ndarray>
    adt        (time, latitude, longitude) float64 dask.array<chunksize=(335, 130, 256), meta=np.ndarray>
    ugos       (time, latitude, longitude) float64 dask.array<chunksize=(335, 130, 256), meta=np.ndarray>
    vgos       (time, latitude, longitude) float64 dask.array<chunksize=(335, 130, 256), meta=np.ndarray>

## Plot MIOST geostrophic currents movie

In [6]:
movie(ds_maps,name_var='uv', method=method_name, region=region, dir_output='../results/',dim_name=['time','latitude','longitude'],
          framerate=24,Display=True,clim=[0,1.2],cmap='YlGnBu_r', newmovie=True)


## Compute and plot DUACS relative vorticity movie

In [7]:
rv = currents_to_relative_vorticity(np.array(ds_maps.ugos),np.array(ds_maps.vgos), ds_maps.longitude,ds_maps.latitude)

ds_maps = ds_maps.assign(rv=lambda ds_maps: ds_maps.sla * 0 + rv)

In [8]:
movie(ds_maps,name_var='rv', method=method_name, region=region, dir_output='../results/',dim_name=['time','latitude','longitude'],
          framerate=24,Display=True,clim=[-0.8,0.8],cmap='RdBu_r', newmovie=True)
